In [1]:
#DataToVid.py
import os
import sys
import numpy as np
import cv2
from PIL import Image

In [2]:
class BinaryString:
    length: int
    data: np.ndarray[bool]
    frame_data: np.ndarray[int] = None
    filename: str
    
    def __init__(self) -> None:
        self.length = 0
        self.data = np.array([], dtype=bool)
    
    def read_file(self, file_path):
        self.filename = os.path.basename(file_path)
        file_size = os.path.getsize(file_path)
        
        idx = 0
        with open(file_path, "rb") as f:
            byte = f.read(1)
            while byte:
                for i in range(8):
                    self.data = np.append(self.data, byte[0] & (1 << i))
                idx += 1
                byte = f.read(1)
                
                percentage = (idx * 100) // file_size
                print(f"\r{idx}/{file_size} ({percentage}%)", end="")
                
        self.length = len(self.data)
    
    def reshape_data(self):
        #reshape data into 1024x1024x3 array of int
        num_frames = self.length // (1024 * 1024 * 3)
        if num_frames == 0:
            num_frames = 1
            #pad data with zeros
            self.data = np.pad(self.data, (0, 1024 * 1024 * 3 - self.length), "constant")

        #[frame, row, col, channel]
        #channel is int 0-255
        self.frame_data = np.zeros((num_frames, 1024, 1024, 3), dtype=int)
        
        #loop through each frame
        #loop through each row
        #loop through each column
        #loop through each channel
        #convert next 8 bits to int
        
        pixel_to_go = num_frames * 1024 * 1024 * 3
        
        pixels_done = 0
        idx = 0
        for frame in range(num_frames):
            for row in range(1024):
                for col in range(1024):
                    for channel in range(3):
                        
                        color_val = 0
                        for i in range(8):
                            if self.data[idx]:
                                color_val |= (1 << i)
                            idx += 1
                        self.frame_data[frame][row][col][channel] = color_val
                        pixels_done += 1
                        print(f"\r{pixels_done}/{pixel_to_go} ({(pixels_done * 100) // pixel_to_go}%)", end="")
                        
        print("Done!")
        
        
        
    def write_file(self):
        
        if not os.path.exists("./data/Output"):
            os.makedirs("./data/Output")
        
        if self.frame_data is None:
            self.reshape_data()
        
        
        output_path = f"./data/Output/{self.filename}.mp4"
        print(f"Writing to {output_path}")
        
        #write to video
        images = []
        
        #convert to PIL images
        for i in range(len(self.frame_data)):
            images.append(Image.fromarray(self.frame_data[i].astype(np.uint8)))
        
        #write to video using cv2
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video = cv2.VideoWriter(output_path, fourcc, 60, (1024, 1024))
        for i in range(len(images)):
            video.write(cv2.cvtColor(np.array(images[i]), cv2.COLOR_RGB2BGR))
        video.release()
        

In [3]:
file_path = "./Data/Input/input.txt"
img = BinaryString()
img.read_file(file_path)

574/574 (100%)

In [4]:
img.reshape_data()

393216/3145728 (12%)

IndexError: index 3145728 is out of bounds for axis 0 with size 3145728

In [ ]:
img.write_file()

Writing to ./data/Output/input.txt.mp4
